In [103]:
import numpy as np
from scipy import signal, io
import matplotlib.pyplot as plt
from acquisition_tools import cal_matrix, estimate_rpm
from tkinter import Tk
from tkinter.filedialog import askopenfilename
%matplotlib qt

plt.rcParams['text.usetex'] = False
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['font.size'] = 16

cal_mat = cal_matrix()

In [104]:
# Load the .mat file
# Open a file dialog to choose a .mat file
root = Tk()
# root.withdraw()  # Hide the main window
file_path = askopenfilename(initialdir='./results', filetypes=[("MAT files", "*.mat")])
root.destroy()  # Destroy the main window

exp_data = io.loadmat(file_path)

In [105]:
time = exp_data['t']
fs_analog = exp_data['Fs_analog'].flatten()[0]

diameter_in = exp_data['D_inch']
pitch_in = exp_data['pitch']

diameter_m = diameter_in * 0.0254
pitch_m = pitch_in * 0.0254

rho = exp_data['rho']

voltage = exp_data['data']
force_net = exp_data['force_net']
force_raw = exp_data['force']
t_analog = np.linspace(0, len(force_raw)/fs_analog, len(force_raw)).flatten()

z_pulse = voltage[:, 6]
z_pulse = np.where(z_pulse > 1, 5, z_pulse)

rpm_sweep = exp_data['rpm_sweep'].flatten()
U_inf = exp_data['U_inf'].flatten()
rpm_command = exp_data['rpm_command'].flatten()

In [106]:
mano_data = exp_data['data_mano']
u = ((mano_data[:, 1] * 200 * 2 / (rho))**0.5).flatten()

In [107]:
indices = np.where(np.diff(z_pulse) > 1)[0]
rpm = 60/np.diff(t_analog.flatten()[indices])
t_rpm = t_analog.flatten()[indices[:-1]]

In [108]:
# Resample rpm to match fs_analog
num_samples = int(len(t_analog))
rpm_resampled = signal.resample(rpm, num_samples)
rpm_command_resampled = signal.resample(rpm_command, num_samples)
u_resampled = signal.resample(u, num_samples)

In [109]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, voltage[:, 0], "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, voltage[:, 1], "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, voltage[:, 2], "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, voltage[:, 3], "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, voltage[:, 4], "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, voltage[:, 5], "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [32]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, force_raw[:, 0], "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, force_raw[:, 1], "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, force_raw[:, 2], "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, force_raw[:, 3], "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, force_raw[:, 4], "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, force_raw[:, 5], "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

In [123]:
dt = 1/fs_analog
order = 2
cutoff = 5
nyq = 0.5 * fs_analog
normal_cutoff = cutoff / nyq

b, a = signal.butter(order, normal_cutoff)
z_l = z_d = signal.lfilter_zi(b, a)


Fx = signal.lfilter(b, a, force_net[:, 0], zi=z_l)[0]
Fy = signal.lfilter(b, a, force_net[:, 1], zi=z_l)[0]
Fz = signal.lfilter(b, a, force_net[:, 2], zi=z_l)[0]
Tx = signal.lfilter(b, a, force_net[:, 3], zi=z_l)[0]
Ty = signal.lfilter(b, a, force_net[:, 4], zi=z_l)[0]
Tz = signal.lfilter(b, a, force_net[:, 5], zi=z_l)[0]

In [77]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].plot(t_analog, Fx, "-")
axes[0, 0].set_xlabel("Time [s]")
axes[0, 0].set_ylabel("$F_x$ [N]")
axes[0, 0].grid()

axes[0, 1].plot(t_analog, Fy, "-")
axes[0, 1].set_xlabel("Time [s]")
axes[0, 1].set_ylabel("$F_y$ [N]")
axes[0, 1].grid()

axes[0, 2].plot(t_analog, Fz, "-")
axes[0, 2].set_xlabel("Time [s]")
axes[0, 2].set_ylabel("$F_z$ [N]")
axes[0, 2].grid()

axes[1, 0].plot(t_analog, Tx, "-")
axes[1, 0].set_xlabel("Time [s]")
axes[1, 0].set_ylabel("$T_x$ [N]")
axes[1, 0].grid()

axes[1, 1].plot(t_analog, Ty, "-")
axes[1, 1].set_xlabel("Time [s]")
axes[1, 1].set_ylabel("$T_y$ [N]")
axes[1, 1].grid()

axes[1, 2].plot(t_analog, Tz, "-")
axes[1, 2].set_xlabel("Time [s]")
axes[1, 2].set_ylabel("$T_z$ [N]")
axes[1, 2].grid()

plt.tight_layout()
plt.show()

Windowing

In [124]:
window_size = 40  # window size in seconds
samples_per_window = int(window_size * fs_analog)  # number of samples per window

# Partition data and t_analog into 40-second windows
t_analog_windows = [t_analog[i:i + samples_per_window] for i in range(0, len(t_analog), samples_per_window) if i + samples_per_window <= len(t_analog)]
Fz_windows = [Fz[i:i + samples_per_window] for i in range(0, len(Fz), samples_per_window) if i + samples_per_window <= len(Fz)]
Tz_windows = [Tz[i:i + samples_per_window] for i in range(0, len(Tz), samples_per_window) if i + samples_per_window <= len(Tz)]
rpm_windows = [rpm_resampled[i:i + samples_per_window] for i in range(0, len(rpm_resampled), samples_per_window) if i + samples_per_window <= len(rpm_resampled)]
u_windows = [u_resampled[i:i + samples_per_window] for i in range(0, len(u_resampled), samples_per_window) if i + samples_per_window <= len(u_resampled)]


# Calculate the mean of each window, leaving out the first 10 seconds
mean_Fz_windows = [np.mean(window[int(10 * fs_analog):]) for window in Fz_windows]
mean_Tz_windows = [np.mean(window[int(10 * fs_analog):]) for window in Tz_windows]
mean_rpm_windows = [np.mean(window[int(10 * fs_analog):]) for window in rpm_windows] # Calculate the mean RPM of each window, leaving out the first 10 seconds
mean_u_windows = [np.mean(window[int(10 * fs_analog):]) for window in u_windows]
mean_n_windows = np.array(mean_rpm_windows) / 60

# Print the mean values of each window
print(mean_rpm_windows)
print(mean_Fz_windows)
print(mean_Tz_windows)
print(mean_u_windows)

[np.float64(4724.673973016577), np.float64(4049.5328815037856), np.float64(3544.0676480842612)]
[np.float64(5.760270396957876), np.float64(3.8057280148751915), np.float64(2.5740455214067404)]
[np.float64(-0.029273568496524908), np.float64(-0.021818579775436053), np.float64(-0.017092909732506316)]
[np.float64(6.028077530881365), np.float64(5.9880090444868515), np.float64(5.991130820629248)]


In [127]:
J = mean_u_windows / (mean_n_windows * diameter_m)
CT = mean_Fz_windows / (rho * mean_n_windows**2 * diameter_m**4)
CQ = -np.array(mean_Tz_windows) / (rho * mean_n_windows**2 * diameter_m**5)
CP = CQ * 2 * np.pi
eta = J * CT / CP

In [ ]:
# Create a figure and two subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# First subplot
ax1.plot(J, CT, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax1.grid()
ax1.set_xlabel('J')
ax1.set_ylabel('$C_T$')
ax1.set_xlim(0.2, 1.0)
# ax1.set_ylim(0.02, 0.12)
ax1.set_xticks(np.arange(0.2, 1.2, 0.2))
# ax1.legend(loc = 'lower left')

# Second subplot
ax2.plot(J, CP, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax2.set_xlim(0.2, 1.0)
# ax2.set_ylim(0.05, 0.1)
ax2.set_xlabel('J')
ax2.set_ylabel('$C_P$')
ax2.set_xticks(np.arange(0.2, 1.2, 0.2))
ax2.grid()

# Third subplot
ax3.plot(J, eta, label='Laminar', color='black', marker='o', linewidth = 2.5)
ax3.set_xlim(0.2, 1.0)
# ax3.set_ylim(0.4, 0.9)
ax3.set_xlabel('J')
ax3.set_ylabel('$\eta$')
ax3.set_xticks(np.arange(0.2, 1.2, 0.2))
ax3.grid()

# Show the plot
plt.tight_layout()
plt.show()

<>:28: SyntaxWarning: invalid escape sequence '\e'
<>:28: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Lab7\AppData\Local\Temp\ipykernel_26044\2076745251.py:28: SyntaxWarning: invalid escape sequence '\e'
  ax3.set_ylabel('$\eta$')


In [129]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# First subplot
ax1.plot(t_analog, Fz, "-", label='Thrust', linewidth = 2, color='k')
ax1.set_ylim(-20, 20)

for i, window in enumerate(Fz_windows):
    start_time = t_analog_windows[i][int(10 * fs_analog)]
    end_time = t_analog_windows[i][-1]
    ax1.axvspan(start_time, end_time, color='gray', alpha=0.3)
    ax2.axvspan(start_time, end_time, color='gray', alpha=0.3)

ax1_twiny = ax1.twinx()
ax1_twiny.plot(t_rpm, rpm, "-", label='RPM', linewidth=2, color='blue')
ax1_twiny.set_ylabel("RPM")

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twiny.get_legend_handles_labels()
ax1_twiny.legend(lines + lines2, labels + labels2, loc='upper left')

ax1.set_xlabel("Time [s]")
ax1.set_ylabel("T [N]")
ax1.grid()

# Second subplot
ax2.plot(t_analog, Tx, "-", label='Torque', linewidth = 2, color='red')
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Q [N.m]")
ax2.legend()
ax2.grid()

plt.tight_layout()
plt.show()

C:\Users\Lab7\AppData\Local\Temp\ipykernel_26044\1651000883.py:32: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.tight_layout()


m:\Efe\TurboProp\venv\Lib\site-packages\ipykernel\eventloops.py:145: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, "exec") else el.exec_()
